In [ ]:
import cv2
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
import threading
import time
from datetime import datetime
import re
from tkinter import PhotoImage
import numpy as np

# Import the model loader from TensorFlow/Keras
from tensorflow.keras.models import load_model

# === Constants for prediction ===
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64   # Preprocessing dimensions for model input
SEQUENCE_LENGTH = 16                 # Number of frames per prediction sequence
CLASSES_LIST = ["NonViolence", "Violence"]

# Define the target camera index which runs the model inference
TARGET_CAMERA = 0

# --- Helper Function ---
def preprocess_frame(frame):
    """Resize and normalize a single frame for model input."""
    frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    frame = frame / 255.0
    return frame

class SmartMonitoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Monitoring & Anomaly Detection")
        self.root.state('zoomed')  # Start in full-screen mode

        # Load background image (update path as necessary)
        self.bg_image_path = "assests/111.jpg"  
        self.bg_image = Image.open(self.bg_image_path)
        self.bg_image = self.bg_image.resize((self.root.winfo_screenwidth(), self.root.winfo_screenheight()))
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Set the background image
        self.bg_label = tk.Label(self.root, image=self.bg_photo)
        self.bg_label.place(relwidth=1, relheight=1)

        # Load users from a file (for login)
        self.users_file = "users.txt"
        self.users = self.load_users()

        # Setup styling for the application
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("TFrame", background="black")
        style.configure("TLabel", background="black", foreground="white")
        style.configure("TEntry", fieldbackground="black", foreground="white")
        style.configure("TButton", background="black", foreground="white")
        self.root.configure(bg="black")

        # Create login frame
        self.login_frame = ttk.Frame(self.root, padding=20, style="TFrame")
        self.login_frame.place(relx=0.5, rely=0.5, anchor="center")

        # Load icons for username and password (update paths as needed)
        self.user_icon = PhotoImage(file="assests/icons8-male-user-50.png")
        self.password_icon = PhotoImage(file="assests/icons8-password-48.png")

        # Username and password fields
        self.username_label = ttk.Label(self.login_frame, text="Username:", font=("Arial", 14))
        self.username_label.grid(row=0, column=0, padx=10, pady=15, sticky="w")
        self.user_icon_label = ttk.Label(self.login_frame, image=self.user_icon, background="black")
        self.user_icon_label.grid(row=0, column=1, padx=(0, 10))
        self.username_entry = ttk.Entry(self.login_frame, font=("Arial", 14))
        self.username_entry.grid(row=0, column=2, padx=(0, 10), pady=15)
        self.password_label = ttk.Label(self.login_frame, text="Password:", font=("Arial", 14))
        self.password_label.grid(row=1, column=0, padx=10, pady=15, sticky="w")
        self.password_icon_label = ttk.Label(self.login_frame, image=self.password_icon, background="black")
        self.password_icon_label.grid(row=1, column=1, padx=(0, 10))
        self.password_entry = ttk.Entry(self.login_frame, show="*", font=("Arial", 14))
        self.password_entry.grid(row=1, column=2, padx=(0, 10), pady=15)
        self.login_button = ttk.Button(self.login_frame, text="Login", command=self.login)
        self.login_button.grid(row=2, column=1, columnspan=2, padx=20, pady=15)

        # Main application frames
        self.main_frame = ttk.Frame(root)
        self.report_frame = ttk.Frame(root)
        self.admin_frame = ttk.Frame(root)

        # Initialize camera captures for 8 cameras.
        # For camera 0 (TARGET_CAMERA): load the specific video for model inference.
        # For all other cameras, set to None (disabled).
        self.num_cameras = 8
        self.captures = []
        for i in range(self.num_cameras):
            if i == TARGET_CAMERA:
                # Load the specific video for camera 0 (update the file path accordingly)
                cap = cv2.VideoCapture("videos/BigFight.mp4")
            else:
                cap = None
            self.captures.append(cap)

        # Create labels to display camera feeds.
        self.camera_labels = []

        # Report Panel (for prediction logs)
        self.report_listbox = tk.Listbox(self.report_frame, width=50, height=15, font=("Arial", 14),
                                         background="black", fg="white")
        self.report_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Admin/Operator Panel components (if needed)
        self.operator_listbox = tk.Listbox(self.admin_frame, width=50, height=15, font=("Arial", 14),
                                           background="black", fg="white")
        self.operator_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        self.add_operator_button = ttk.Button(self.admin_frame, text="Add Operator", command=self.add_operator)
        self.add_operator_button.pack(padx=10, pady=10)
        self.delete_operator_button = ttk.Button(self.admin_frame, text="Delete Operator", command=self.delete_operator)
        self.delete_operator_button.pack(padx=10, pady=10)

        # "Go Back" button to return to the login interface.
        self.go_back_button = ttk.Button(self.root, text="Go Back", command=self.go_back)
        self.go_back_button.pack(padx=10, pady=10)

        # Load the model (MobileNetV2+biLSTM) from file.
        try:
            self.model = load_model("models/mobileNetv2_biLSTM.h5")
        except Exception as e:
            messagebox.showerror("Model Load Error", f"Failed to load model: {e}")
            self.model = None

        # Variables for prediction logging and frame buffering.
        self.last_prediction = None
        self.last_report_time = 0
        self.frames_buffer = []  # Buffer for the last SEQUENCE_LENGTH preprocessed frames

    def load_users(self):
        """Loads users from file into a dictionary."""
        users = {}
        try:
            with open(self.users_file, "r") as file:
                for line in file:
                    username, password, role = line.strip().split(",")
                    users[username] = {"password": password, "role": role}
        except FileNotFoundError:
            with open(self.users_file, "w") as file:
                file.write("admin,admin123,admin\n")
            users = {"admin": {"password": "admin123", "role": "admin"}}
        return users

    def save_user(self, username, password, role):
        """Saves a new user to file."""
        with open(self.users_file, "a") as file:
            file.write(f"{username},{password},{role}\n")

    def delete_user(self, username):
        """Deletes a user from file."""
        with open(self.users_file, "r") as file:
            lines = file.readlines()
        with open(self.users_file, "w") as file:
            for line in lines:
                if not line.startswith(username + ","):
                    file.write(line)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        if not re.match("^[A-Za-z]+$", username):
            messagebox.showerror("Invalid Username", "Username must contain only letters.")
            return
        if len(password) < 8:
            messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
            return
        if username in self.users and self.users[username]["password"] == password:
            self.login_frame.destroy()
            if self.users[username]["role"] == "admin":
                self.show_admin_interface()
            else:
                self.show_operator_interface()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password")

    def add_operator(self):
        username = simpledialog.askstring("Add Operator", "Enter username:")
        if username:
            if not re.match("^[A-Za-z]+$", username):
                messagebox.showerror("Invalid Username", "Username must contain only letters.")
                return
            if username in self.users:
                messagebox.showerror("Error", "Username already exists!")
                return
            password = simpledialog.askstring("Add Operator", "Enter password:", show="*")
            if password:
                if len(password) < 8:
                    messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
                    return
                self.save_user(username, password, "operator")
                self.users[username] = {"password": password, "role": "operator"}
                self.update_operator_listbox()
                messagebox.showinfo("Success", f"Operator '{username}' added successfully!")
            else:
                messagebox.showerror("Error", "Password cannot be empty!")
        else:
            messagebox.showerror("Error", "Username cannot be empty!")

    def update_operator_listbox(self):
        self.operator_listbox.delete(0, tk.END)
        for username, info in self.users.items():
            if info["role"] == "operator":
                self.operator_listbox.insert(tk.END, username)

    def delete_operator(self):
        selected = self.operator_listbox.curselection()
        if selected:
            username = self.operator_listbox.get(selected)
            if username in self.users:
                self.delete_user(username)
                del self.users[username]
                self.update_operator_listbox()
                messagebox.showinfo("Success", f"Operator '{username}' deleted successfully!")
            else:
                messagebox.showerror("Error", "Operator not found!")
        else:
            messagebox.showerror("Error", "No operator selected!")

    def show_admin_interface(self):
        self.admin_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
        self.update_operator_listbox()

    def show_operator_interface(self):
        self.main_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
        self.create_camera_grid()
        self.start_video_threads()

    def create_camera_grid(self):
        """Creates a grid layout for displaying camera feeds."""
        rows, cols = 4, 2
        for i in range(self.num_cameras):
            frame = ttk.LabelFrame(self.main_frame, text=f"Camera {i+1}")
            frame.grid(row=i // cols, column=i % cols, padx=10, pady=10, sticky="nsew")
            label = tk.Label(frame, text="Initializing...", font=("Arial", 14), fg="red")
            label.pack(fill=tk.BOTH, expand=True)
            self.camera_labels.append(label)
        for i in range(rows):
            self.main_frame.grid_rowconfigure(i, weight=1)
        for j in range(cols):
            self.main_frame.grid_columnconfigure(j, weight=1)

    def start_video_threads(self):
        """
        Starts the appropriate thread for each camera:
        - Camera 0 (TARGET_CAMERA) runs update_camera (with model inference using the specific video).
        - All other cameras are disabled.
        """
        for i in range(self.num_cameras):
            if i == TARGET_CAMERA:
                threading.Thread(target=self.update_camera, args=(i,), daemon=True).start()
            else:
                self.camera_labels[i].config(text="Camera Disabled", font=("Arial", 16), fg="yellow")

    def predict_violence(self, frames_list):
        """
        Runs model prediction over the last SEQUENCE_LENGTH frames.
        Returns the predicted class label.
        """
        input_frames = np.array([frames_list[-SEQUENCE_LENGTH:]])
        prediction = self.model.predict(input_frames)
        predicted_class = np.argmax(prediction)
        return CLASSES_LIST[predicted_class]

    def update_camera(self, index):
        """
        Reads frames from camera 0 (the specific video),
        buffers them for sequence prediction,
        overlays prediction text on the display frame,
        and logs the result to the report panel.
        
        *Note:* Predictions with label "NonViolence" are not logged.
        """
        if self.captures[index] is None:
            return

        while True:
            ret, frame = self.captures[index].read()
            if not ret:
                # Restart video if we reach the end.
                self.captures[index].set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue

            # Convert BGR to RGB and resize for display.
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            display_frame = cv2.resize(rgb_frame, (400, 300))
            
            # Preprocess the frame for prediction.
            proc_frame = preprocess_frame(rgb_frame)
            self.frames_buffer.append(proc_frame)
            if len(self.frames_buffer) > SEQUENCE_LENGTH:
                self.frames_buffer = self.frames_buffer[-SEQUENCE_LENGTH:]

            # Run prediction if enough frames exist.
            if len(self.frames_buffer) >= SEQUENCE_LENGTH and self.model is not None:
                pred_label = self.predict_violence(self.frames_buffer)
                # cv2.putText(display_frame, f"Prediction: {pred_label}", (10, 25),
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                current_time = time.time()
                # Only log predictions if they are not "NonViolence"
                # and if the prediction changed or 5 seconds have elapsed.
                if pred_label != "NonViolence" and (self.last_prediction != pred_label or (current_time - self.last_report_time > 5)):
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    report = f"[{timestamp}] Prediction: {pred_label}"
                    self.report_listbox.insert(tk.END, report)
                    self.report_listbox.insert(tk.END, "-" * 50)
                    self.last_prediction = pred_label
                    self.last_report_time = current_time
            else:
                cv2.putText(display_frame, "Loading...", (10, 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            img = ImageTk.PhotoImage(image=Image.fromarray(display_frame))
            self.camera_labels[index].imgtk = img
            self.camera_labels[index].config(image=img)
            time.sleep(0.03)

    def go_back(self):
        """Resets the UI and returns to the login page."""
        for widget in self.root.winfo_children():
            widget.destroy()
        self.__init__(self.root)

    def __del__(self):
        if hasattr(self, 'captures'):
            for cap in self.captures:
                if cap and cap.isOpened():
                    cap.release()

if __name__ == "__main__":
    root = tk.Tk()
    app = SmartMonitoringApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━

In [ ]:
# import cv2
# import tkinter as tk
# from tkinter import ttk, messagebox, simpledialog
# from PIL import Image, ImageTk
# import threading
# import time
# from datetime import datetime
# import re
# from tkinter import PhotoImage
# import numpy as np
# from tensorflow.keras.models import load_model
# import pandas as pd
# from ultralytics import YOLO

# # Constants for prediction
# IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64   # Preprocessing dimensions for sequence model input
# SEQUENCE_LENGTH = 16                 # Number of frames per prediction sequence for sequence model
# CLASSES_LIST = ["NonViolence", "Violence"]

# # Helper Function
# def preprocess_frame(frame):
#     """Resize and normalize a single frame for sequence model input."""
#     frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
#     frame = frame / 255.0
#     return frame

# class SmartMonitoringApp:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("Smart Monitoring & Anomaly Detection")
#         self.root.state('zoomed')  # Start in full-screen mode

#         # Load background image (update path as necessary)
#         self.bg_image_path = "assests/111.jpg"  
#         self.bg_image = Image.open(self.bg_image_path)
#         self.bg_image = self.bg_image.resize((self.root.winfo_screenwidth(), self.root.winfo_screenheight()))
#         self.bg_photo = ImageTk.PhotoImage(self.bg_image)

#         # Set the background image
#         self.bg_label = tk.Label(self.root, image=self.bg_photo)
#         self.bg_label.place(relwidth=1, relheight=1)

#         # Load users from a file (for login)
#         self.users_file = "users.txt"
#         self.users = self.load_users()

#         # Setup styling for the application
#         style = ttk.Style()
#         style.theme_use('clam')
#         style.configure("TFrame", background="black")
#         style.configure("TLabel", background="black", foreground="white")
#         style.configure("TEntry", fieldbackground="black", foreground="white")
#         style.configure("TButton", background="black", foreground="white")
#         self.root.configure(bg="black")

#         # Create login frame
#         self.login_frame = ttk.Frame(self.root, padding=20, style="TFrame")
#         self.login_frame.place(relx=0.5, rely=0.5, anchor="center")

#         # Load icons for username and password (update paths as needed)
#         self.user_icon = PhotoImage(file="assests/icons8-male-user-50.png")
#         self.password_icon = PhotoImage(file="assests/icons8-password-48.png")

#         # Username and password fields
#         self.username_label = ttk.Label(self.login_frame, text="Username:", font=("Arial", 14))
#         self.username_label.grid(row=0, column=0, padx=10, pady=15, sticky="w")
#         self.user_icon_label = ttk.Label(self.login_frame, image=self.user_icon, background="black")
#         self.user_icon_label.grid(row=0, column=1, padx=(0, 10))
#         self.username_entry = ttk.Entry(self.login_frame, font=("Arial", 14))
#         self.username_entry.grid(row=0, column=2, padx=(0, 10), pady=15)
#         self.password_label = ttk.Label(self.login_frame, text="Password:", font=("Arial", 14))
#         self.password_label.grid(row=1, column=0, padx=10, pady=15, sticky="w")
#         self.password_icon_label = ttk.Label(self.login_frame, image=self.password_icon, background="black")
#         self.password_icon_label.grid(row=1, column=1, padx=(0, 10))
#         self.password_entry = ttk.Entry(self.login_frame, show="*", font=("Arial", 14))
#         self.password_entry.grid(row=1, column=2, padx=(0, 10), pady=15)
#         self.login_button = ttk.Button(self.login_frame, text="Login", command=self.login)
#         self.login_button.grid(row=2, column=1, columnspan=2, padx=20, pady=15)

#         # Main application frames
#         self.main_frame = ttk.Frame(root)
#         self.report_frame = ttk.Frame(root)
#         self.admin_frame = ttk.Frame(root)

#         # Initialize camera captures and models for 8 cameras
#         self.num_cameras = 8
#         self.captures = []
#         self.models = [None] * self.num_cameras
#         self.model_types = [None] * self.num_cameras
#         self.class_list = None

#         # Load models and set model types
#         for i in range(self.num_cameras):
#             if i == 0:
#                 try:
#                     self.models[i] = load_model("models/mobileNetv2_biLSTM.h5")
#                     self.model_types[i] = "sequence"
#                 except Exception as e:
#                     messagebox.showerror("Model Load Error", f"Failed to load model for camera 0: {e}")
#             elif i == 1:
#                 try:
#                     self.models[i] = YOLO("models/best.pt")  # Load YOLO model as in the snippet
#                     self.model_types[i] = "yolo"
#                     with open("models/coco1.txt", "r") as my_file:
#                         data = my_file.read()
#                         self.class_list = data.split("\n")
#                 except Exception as e:
#                     messagebox.showerror("Model Load Error", f"Failed to load YOLO model for camera 1: {e}")
#             else:
#                 self.captures.append(None)

#         # Set up captures
#         for i in range(self.num_cameras):
#             if i == 0:
#                 cap = cv2.VideoCapture("videos/BigFight.mp4")
#                 if not cap.isOpened():
#                     print(f"Error: Could not open 'videos/BigFight.mp4' for Camera 1")
#                     cap = None
#             elif i == 1:
#                 cap = cv2.VideoCapture("videos/cr.mp4")
#                 if not cap.isOpened():
#                     print(f"Error: Could not open 'videos/carnorm.mp4' for Camera 2")
#                     cap = None
#             else:
#                 cap = None
#             self.captures.append(cap)

#         # Create labels to display camera feeds
#         self.camera_labels = []

#         # Report Panel (for prediction logs)
#         self.report_listbox = tk.Listbox(self.report_frame, width=50, height=15, font=("Arial", 14),
#                                          background="black", fg="white")
#         self.report_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

#         # Admin/Operator Panel components
#         self.operator_listbox = tk.Listbox(self.admin_frame, width=50, height=15, font=("Arial", 14),
#                                            background="black", fg="white")
#         self.operator_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
#         self.add_operator_button = ttk.Button(self.admin_frame, text="Add Operator", command=self.add_operator)
#         self.add_operator_button.pack(padx=10, pady=10)
#         self.delete_operator_button = ttk.Button(self.admin_frame, text="Delete Operator", command=self.delete_operator)
#         self.delete_operator_button.pack(padx=10, pady=10)

#         # "Go Back" button
#         self.go_back_button = ttk.Button(self.root, text="Go Back", command=self.go_back)
#         self.go_back_button.pack(padx=10, pady=10)

#         # Per-camera buffers and logging variables
#         self.frames_buffers = [[] for _ in range(self.num_cameras)]
#         self.last_predictions = [None] * self.num_cameras
#         self.last_report_times = [0] * self.num_cameras

#     def load_users(self):
#         """Loads users from file into a dictionary."""
#         users = {}
#         try:
#             with open(self.users_file, "r") as file:
#                 for line in file:
#                     username, password, role = line.strip().split(",")
#                     users[username] = {"password": password, "role": role}
#         except FileNotFoundError:
#             with open(self.users_file, "w") as file:
#                 file.write("admin,admin123,admin\n")
#             users = {"admin": {"password": "admin123", "role": "admin"}}
#         return users

#     def save_user(self, username, password, role):
#         """Saves a new user to file."""
#         with open(self.users_file, "a") as file:
#             file.write(f"{username},{password},{role}\n")

#     def delete_user(self, username):
#         """Deletes a user from file."""
#         with open(self.users_file, "r") as file:
#             lines = file.readlines()
#         with open(self.users_file, "w") as file:
#             for line in lines:
#                 if not line.startswith(username + ","):
#                     file.write(line)

#     def login(self):
#         username = self.username_entry.get()
#         password = self.password_entry.get()
#         if not re.match("^[A-Za-z]+$", username):
#             messagebox.showerror("Invalid Username", "Username must contain only letters.")
#             return
#         if len(password) < 8:
#             messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
#             return
#         if username in self.users and self.users[username]["password"] == password:
#             self.login_frame.destroy()
#             if self.users[username]["role"] == "admin":
#                 self.show_admin_interface()
#             else:
#                 self.show_operator_interface()
#         else:
#             messagebox.showerror("Login Failed", "Invalid username or password")

#     def add_operator(self):
#         username = simpledialog.askstring("Add Operator", "Enter username:")
#         if username:
#             if not re.match("^[A-Za-z]+$", username):
#                 messagebox.showerror("Invalid Username", "Username must contain only letters.")
#                 return
#             if username in self.users:
#                 messagebox.showerror("Error", "Username already exists!")
#                 return
#             password = simpledialog.askstring("Add Operator", "Enter password:", show="*")
#             if password:
#                 if len(password) < 8:
#                     messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
#                     return
#                 self.save_user(username, password, "operator")
#                 self.users[username] = {"password": password, "role": "operator"}
#                 self.update_operator_listbox()
#                 messagebox.showinfo("Success", f"Operator '{username}' added successfully!")
#             else:
#                 messagebox.showerror("Error", "Password cannot be empty!")
#         else:
#             messagebox.showerror("Error", "Username cannot be empty!")

#     def update_operator_listbox(self):
#         self.operator_listbox.delete(0, tk.END)
#         for username, info in self.users.items():
#             if info["role"] == "operator":
#                 self.operator_listbox.insert(tk.END, username)

#     def delete_operator(self):
#         selected = self.operator_listbox.curselection()
#         if selected:
#             username = self.operator_listbox.get(selected)
#             if username in self.users:
#                 self.delete_user(username)
#                 del self.users[username]
#                 self.update_operator_listbox()
#                 messagebox.showinfo("Success", f"Operator '{username}' deleted successfully!")
#             else:
#                 messagebox.showerror("Error", "Operator not found!")
#         else:
#             messagebox.showerror("Error", "No operator selected!")

#     def show_admin_interface(self):
#         self.admin_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
#         self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
#         self.update_operator_listbox()

#     def show_operator_interface(self):
#         self.main_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
#         self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
#         self.create_camera_grid()
#         self.start_video_threads()

#     def create_camera_grid(self):
#         """Creates a grid layout for displaying camera feeds."""
#         rows, cols = 4, 2
#         for i in range(self.num_cameras):
#             frame = ttk.LabelFrame(self.main_frame, text=f"Camera {i+1}")
#             frame.grid(row=i // cols, column=i % cols, padx=10, pady=10, sticky="nsew")
#             label = tk.Label(frame, text="Initializing...", font=("Arial", 14), fg="red")
#             label.pack(fill=tk.BOTH, expand=True)
#             self.camera_labels.append(label)
#         for i in range(rows):
#             self.main_frame.grid_rowconfigure(i, weight=1)
#         for j in range(cols):
#             self.main_frame.grid_columnconfigure(j, weight=1)

#     def start_video_threads(self):
#         """Starts threads for cameras with active captures."""
#         for i in range(self.num_cameras):
#             if self.captures[i] is not None:
#                 print(f"Starting thread for Camera {i+1} (index {i})")
#                 threading.Thread(target=self.update_camera, args=(i,), daemon=True).start()
#             else:
#                 print(f"Setting Camera {i+1} (index {i}) to 'Camera Disabled'")
#                 self.camera_labels[i].config(text="Camera Disabled", font=("Arial", 16), fg="yellow")

#     def predict_violence(self, frames_list, model):
#         """Runs model prediction for sequence-based models."""
#         input_frames = np.array([frames_list[-SEQUENCE_LENGTH:]])
#         prediction = model.predict(input_frames)
#         predicted_class = np.argmax(prediction)
#         return CLASSES_LIST[predicted_class]

#     def update_camera(self, index):
#         """Updates the camera feed and runs predictions if applicable."""
#         if self.captures[index] is None or not self.captures[index].isOpened():
#             self.camera_labels[index].config(text="Camera Disabled", font=("Arial", 16), fg="yellow")
#             return

#         count = 0  # Frame counter for YOLO processing
#         while True:
#             ret, frame = self.captures[index].read()
#             if not ret:
#                 self.captures[index].set(cv2.CAP_PROP_POS_FRAMES, 0)  # Loop video
#                 continue

#             rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#             if self.model_types[index] == "sequence":
#                 display_frame = cv2.resize(rgb_frame, (400, 300))
#                 proc_frame = preprocess_frame(rgb_frame)
#                 self.frames_buffers[index].append(proc_frame)
#                 if len(self.frames_buffers[index]) > SEQUENCE_LENGTH:
#                     self.frames_buffers[index] = self.frames_buffers[index][-SEQUENCE_LENGTH:]
#                 if len(self.frames_buffers[index]) >= SEQUENCE_LENGTH and self.models[index] is not None:
#                     pred_label = self.predict_violence(self.frames_buffers[index], self.models[index])
#                 else:
#                     pred_label = "Loading..."
#                 cv2.putText(display_frame, f"Prediction: {pred_label}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

#             elif self.model_types[index] == "yolo":
#                 count += 1
#                 if count % 3 != 0:  # Process every 3rd frame as in the snippet
#                     continue
#                 frame_for_processing = cv2.resize(rgb_frame, (1020, 500))  # Resize as in the snippet
#                 results = self.models[index].predict(frame_for_processing)
#                 a = results[0].boxes.data
#                 px = pd.DataFrame(a).astype("float")
#                 for index_row, row in px.iterrows():
#                     x1 = int(row[0])
#                     y1 = int(row[1])
#                     x2 = int(row[2])
#                     y2 = int(row[3])
#                     confidence = float(row[4])
#                     class_id = int(row[5])
#                     if confidence < 0.60:  # Confidence threshold as in the snippet
#                         continue
#                     c = self.class_list[class_id]
#                     cv2.rectangle(frame_for_processing, (x1, y1), (x2, y2), (0, 255, 0), 1)
#                     # Use cv2.putText instead of cvzone to match dependencies
#                     cv2.putText(frame_for_processing, f'{c} {confidence:.2f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
#                 display_frame = cv2.resize(frame_for_processing, (400, 300))

#             # Convert to ImageTk
#             img = ImageTk.PhotoImage(image=Image.fromarray(display_frame))
#             self.camera_labels[index].imgtk = img
#             self.camera_labels[index].config(image=img)

#             time.sleep(0.03)

#     def go_back(self):
#         """Resets the UI and returns to the login page."""
#         for widget in self.root.winfo_children():
#             widget.destroy()
#         self.__init__(self.root)

#     def __del__(self):
#         if hasattr(self, 'captures'):
#             for cap in self.captures:
#                 if cap and cap.isOpened():
#                     cap.release()

# if __name__ == "__main__":
#     root = tk.Tk()
#     app = SmartMonitoringApp(root)
#     root.mainloop()

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Users\medoo\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "C:\Users\medoo\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\medoo\AppData\Local\Programs\Python\Python39\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\medoo\AppData\Local\Temp\ipykernel_36240\3271281467.py", line 336, in update_camera
UnboundLocalError: local variable 'display_frame' referenced before assignment
Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\Users\medoo\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "C:\Users\medoo\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run

Setting Camera 1 (index 0) to 'Camera Disabled'
Setting Camera 2 (index 1) to 'Camera Disabled'
Setting Camera 3 (index 2) to 'Camera Disabled'
Setting Camera 4 (index 3) to 'Camera Disabled'
Setting Camera 5 (index 4) to 'Camera Disabled'
Setting Camera 6 (index 5) to 'Camera Disabled'
Starting thread for Camera 7 (index 6)
Starting thread for Camera 8 (index 7)


In [2]:
import cv2
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
import threading
import time
from datetime import datetime
import re
from tkinter import PhotoImage
import numpy as np

# Import for model1 (MobileNetV2+biLSTM)
from tensorflow.keras.models import load_model

# Import YOLO model from Ultralytics for model2
from ultralytics import YOLO

# === Constants for model1 prediction (MobileNetV2+biLSTM) ===
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64   # Preprocessing dimensions for model1 input
SEQUENCE_LENGTH = 16                 # Number of frames per prediction sequence
CLASSES_LIST = ["NonViolence", "Violence"]

# Define target camera indices
TARGET_CAMERA_MODEL1 = 0   # For model1 (MobileNetV2+biLSTM)
TARGET_CAMERA_YOLO   = 1   # For model2 (YOLO)

# --- Helper Function for model1 ---
def preprocess_frame(frame):
    """Resize and normalize a single frame for model1 input."""
    frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    frame = frame / 255.0
    return frame

class SmartMonitoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Monitoring & Anomaly Detection")
        self.root.state('zoomed')  # Start in full-screen mode

        # Load background image (update path as necessary)
        self.bg_image_path = "assests/111.jpg"  
        self.bg_image = Image.open(self.bg_image_path)
        self.bg_image = self.bg_image.resize((self.root.winfo_screenwidth(), self.root.winfo_screenheight()))
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)
        self.bg_label = tk.Label(self.root, image=self.bg_photo)
        self.bg_label.place(relwidth=1, relheight=1)

        # Load users from a file (for login)
        self.users_file = "users.txt"
        self.users = self.load_users()

        # Setup styling for the application
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("TFrame", background="black")
        style.configure("TLabel", background="black", foreground="white")
        style.configure("TEntry", fieldbackground="black", foreground="white")
        style.configure("TButton", background="black", foreground="white")
        self.root.configure(bg="black")

        # Create login frame
        self.login_frame = ttk.Frame(self.root, padding=20, style="TFrame")
        self.login_frame.place(relx=0.5, rely=0.5, anchor="center")

        # Load icons for username and password (update paths as needed)
        self.user_icon = PhotoImage(file="assests/icons8-male-user-50.png")
        self.password_icon = PhotoImage(file="assests/icons8-password-48.png")

        # Username and password fields
        self.username_label = ttk.Label(self.login_frame, text="Username:", font=("Arial", 14))
        self.username_label.grid(row=0, column=0, padx=10, pady=15, sticky="w")
        self.user_icon_label = ttk.Label(self.login_frame, image=self.user_icon, background="black")
        self.user_icon_label.grid(row=0, column=1, padx=(0, 10))
        self.username_entry = ttk.Entry(self.login_frame, font=("Arial", 14))
        self.username_entry.grid(row=0, column=2, padx=(0, 10), pady=15)
        self.password_label = ttk.Label(self.login_frame, text="Password:", font=("Arial", 14))
        self.password_label.grid(row=1, column=0, padx=10, pady=15, sticky="w")
        self.password_icon_label = ttk.Label(self.login_frame, image=self.password_icon, background="black")
        self.password_icon_label.grid(row=1, column=1, padx=(0, 10))
        self.password_entry = ttk.Entry(self.login_frame, show="*", font=("Arial", 14))
        self.password_entry.grid(row=1, column=2, padx=(0, 10), pady=15)
        self.login_button = ttk.Button(self.login_frame, text="Login", command=self.login)
        self.login_button.grid(row=2, column=1, columnspan=2, padx=20, pady=15)

        # Main application frames
        self.main_frame = ttk.Frame(root)
        self.report_frame = ttk.Frame(root)
        self.admin_frame = ttk.Frame(root)

        # Initialize camera captures for 8 cameras.
        # For camera 0: load video for model1; for camera 1: load video for model2.
        self.num_cameras = 8
        self.captures = []
        for i in range(self.num_cameras):
            if i == TARGET_CAMERA_MODEL1:
                # Video for model1 on camera 0 (update file path accordingly)
                cap = cv2.VideoCapture("videos/BigFight.mp4")
            elif i == TARGET_CAMERA_YOLO:
                # Video for YOLO on camera 1 (update file path accordingly)
                cap = cv2.VideoCapture("videos/cr.mp4")
            else:
                cap = None
            self.captures.append(cap)

        # Create labels to display camera feeds.
        self.camera_labels = []

        # Report Panel (for logging predictions/detections)
        self.report_listbox = tk.Listbox(self.report_frame, width=50, height=15, font=("Arial", 14),
                                         background="black", fg="white")
        self.report_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Admin/Operator Panel components (if needed)
        self.operator_listbox = tk.Listbox(self.admin_frame, width=50, height=15, font=("Arial", 14),
                                           background="black", fg="white")
        self.operator_listbox.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        self.add_operator_button = ttk.Button(self.admin_frame, text="Add Operator", command=self.add_operator)
        self.add_operator_button.pack(padx=10, pady=10)
        self.delete_operator_button = ttk.Button(self.admin_frame, text="Delete Operator", command=self.delete_operator)
        self.delete_operator_button.pack(padx=10, pady=10)

        # "Go Back" button to return to the login interface.
        self.go_back_button = ttk.Button(self.root, text="Go Back", command=self.go_back)
        self.go_back_button.pack(padx=10, pady=10)

        # Load model1 (MobileNetV2+biLSTM) from file.
        try:
            self.model = load_model("models/mobileNetv2_biLSTM.h5")
        except Exception as e:
            messagebox.showerror("Model1 Load Error", f"Failed to load model1: {e}")
            self.model = None

        # Load model2 (YOLO) using Ultralytics.
        try:
            self.yolo_model = YOLO('models/best.pt')
            # YOLO model from Ultralytics typically provides model.names (a dict mapping class indices to names)
            self.yolo_names = self.yolo_model.names  # This may be a dict: {0: 'person', 1: 'bicycle', ...}
        except Exception as e:
            messagebox.showerror("YOLO Model Load Error", f"Failed to load YOLO model: {e}")
            self.yolo_model = None
            self.yolo_names = {}

        # Variables for model1 prediction logging and frame buffering.
        self.last_prediction = None
        self.last_report_time = 0
        self.frames_buffer = []  # Buffer for the last SEQUENCE_LENGTH preprocessed frames
        # Variables for YOLO model logging.
        self.last_yolo_report_time = 0
        self.last_yolo_detections = None

    def load_users(self):
        """Loads users from file into a dictionary."""
        users = {}
        try:
            with open(self.users_file, "r") as file:
                for line in file:
                    username, password, role = line.strip().split(",")
                    users[username] = {"password": password, "role": role}
        except FileNotFoundError:
            with open(self.users_file, "w") as file:
                file.write("admin,admin123,admin\n")
            users = {"admin": {"password": "admin123", "role": "admin"}}
        return users

    def save_user(self, username, password, role):
        """Saves a new user to file."""
        with open(self.users_file, "a") as file:
            file.write(f"{username},{password},{role}\n")

    def delete_user(self, username):
        """Deletes a user from file."""
        with open(self.users_file, "r") as file:
            lines = file.readlines()
        with open(self.users_file, "w") as file:
            for line in lines:
                if not line.startswith(username + ","):
                    file.write(line)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        if not re.match("^[A-Za-z]+$", username):
            messagebox.showerror("Invalid Username", "Username must contain only letters.")
            return
        if len(password) < 8:
            messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
            return
        if username in self.users and self.users[username]["password"] == password:
            self.login_frame.destroy()
            if self.users[username]["role"] == "admin":
                self.show_admin_interface()
            else:
                self.show_operator_interface()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password")

    def add_operator(self):
        username = simpledialog.askstring("Add Operator", "Enter username:")
        if username:
            if not re.match("^[A-Za-z]+$", username):
                messagebox.showerror("Invalid Username", "Username must contain only letters.")
                return
            if username in self.users:
                messagebox.showerror("Error", "Username already exists!")
                return
            password = simpledialog.askstring("Add Operator", "Enter password:", show="*")
            if password:
                if len(password) < 8:
                    messagebox.showerror("Invalid Password", "Password must be at least 8 characters long.")
                    return
                self.save_user(username, password, "operator")
                self.users[username] = {"password": password, "role": "operator"}
                self.update_operator_listbox()
                messagebox.showinfo("Success", f"Operator '{username}' added successfully!")
            else:
                messagebox.showerror("Error", "Password cannot be empty!")
        else:
            messagebox.showerror("Error", "Username cannot be empty!")

    def update_operator_listbox(self):
        self.operator_listbox.delete(0, tk.END)
        for username, info in self.users.items():
            if info["role"] == "operator":
                self.operator_listbox.insert(tk.END, username)

    def delete_user_from_list(self):
        selected = self.operator_listbox.curselection()
        if selected:
            username = self.operator_listbox.get(selected)
            if username in self.users:
                self.delete_user(username)
                del self.users[username]
                self.update_operator_listbox()
                messagebox.showinfo("Success", f"Operator '{username}' deleted successfully!")
            else:
                messagebox.showerror("Error", "Operator not found!")
        else:
            messagebox.showerror("Error", "No operator selected!")

    def delete_operator(self):
        self.delete_user_from_list()

    def show_admin_interface(self):
        self.admin_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
        self.update_operator_listbox()

    def show_operator_interface(self):
        self.main_frame.pack(padx=20, pady=20, side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.report_frame.pack(padx=20, pady=20, side=tk.RIGHT, fill=tk.BOTH, expand=True)
        self.create_camera_grid()
        self.start_video_threads()

    def create_camera_grid(self):
        """Creates a grid layout for displaying camera feeds."""
        rows, cols = 4, 2
        for i in range(self.num_cameras):
            frame = ttk.LabelFrame(self.main_frame, text=f"Camera {i+1}")
            frame.grid(row=i // cols, column=i % cols, padx=10, pady=10, sticky="nsew")
            label = tk.Label(frame, text="Initializing...", font=("Arial", 14), fg="red")
            label.pack(fill=tk.BOTH, expand=True)
            self.camera_labels.append(label)
        for i in range(rows):
            self.main_frame.grid_rowconfigure(i, weight=1)
        for j in range(cols):
            self.main_frame.grid_columnconfigure(j, weight=1)

    def start_video_threads(self):
        """
        Starts the appropriate thread for each camera:
        - Camera 0 runs update_camera (model1 inference on specific video).
        - Camera 1 runs update_camera_yolo (YOLO detection on specific video).
        - All other cameras are disabled.
        """
        for i in range(self.num_cameras):
            if i == TARGET_CAMERA_MODEL1:
                threading.Thread(target=self.update_camera, args=(i,), daemon=True).start()
            elif i == TARGET_CAMERA_YOLO:
                threading.Thread(target=self.update_camera_yolo, args=(i,), daemon=True).start()
            else:
                self.camera_labels[i].config(text="Camera Disabled", font=("Arial", 16), fg="yellow")

    def predict_violence(self, frames_list):
        """
        Runs model1 prediction over the last SEQUENCE_LENGTH frames.
        Returns the predicted class label.
        """
        input_frames = np.array([frames_list[-SEQUENCE_LENGTH:]])
        prediction = self.model.predict(input_frames)
        predicted_class = np.argmax(prediction)
        return CLASSES_LIST[predicted_class]

    def update_camera(self, index):
        """
        Reads frames from camera 0 (video for model1),
        buffers them for sequence prediction,
        and logs the result to the report panel if the predicted label is not "NonViolence".
        """
        if self.captures[index] is None:
            return

        while True:
            ret, frame = self.captures[index].read()
            if not ret:
                # Restart video if reached end.
                self.captures[index].set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue

            # Process frame: convert BGR to RGB and resize for display.
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            display_frame = cv2.resize(rgb_frame, (400, 300))
            
            # Preprocess frame for model1.
            proc_frame = preprocess_frame(rgb_frame)
            self.frames_buffer.append(proc_frame)
            # if len(self.frames_buffer) > SEQUENCE_LENGTH:
            self.frames_buffer = self.frames_buffer[-SEQUENCE_LENGTH:]

            # Run model1 prediction if enough frames.
            # if len(self.frames_buffer) >= SEQUENCE_LENGTH and self.model is not None:
            pred_label = self.predict_violence(self.frames_buffer)
            current_time = time.time()
            # Only log if prediction is not "NonViolence"
            if pred_label != "NonViolence" and (self.last_prediction != pred_label or (current_time - self.last_report_time > 5)):
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                report = f"[{timestamp}] Prediction (Model1): {pred_label}"
                self.report_listbox.insert(tk.END, report)
                self.report_listbox.insert(tk.END, "-" * 50)
                self.last_prediction = pred_label
                self.last_report_time = current_time
            # else:
            #     cv2.putText(display_frame, "Loading...", (10, 25),
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            img = ImageTk.PhotoImage(image=Image.fromarray(display_frame))
            self.camera_labels[index].imgtk = img
            self.camera_labels[index].config(image=img)
            time.sleep(0.03)

    def update_camera_yolo(self, index):
        """
        Reads frames from camera 1 (video for YOLO),
        runs object detection using the YOLO model (Ultralytics),
        overlays bounding boxes and labels on the frame,
        and logs detections to the report panel.
        Only detections with confidence >= 0.80 are considered.
        """
        if self.captures[index] is None or self.yolo_model is None:
            return

        confidence_threshold = 0.80

        while True:
            ret, frame = self.captures[index].read()
            if not ret:
                self.captures[index].set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue

            # Run YOLO detection (Ultralytics returns a list of results for each image)
            results = self.yolo_model(frame)[0]  # Get detections for this frame
            annotated_frame = frame.copy()
            current_detections = []  # List to store detection strings for this frame

            # Loop through detected objects
            for box in results.boxes:
                conf = float(box.conf.cpu().numpy()[0])
                # Skip if confidence is below threshold
                if conf < confidence_threshold:
                    continue
                xyxy = box.xyxy.cpu().numpy()[0].astype(int)
                cls = int(box.cls.cpu().numpy()[0])
                label = self.yolo_names.get(cls, str(cls))
                current_detections.append(f"{label} {conf:.2f}")
                # Draw bounding box and label
                # cv2.rectangle(annotated_frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
                # cv2.putText(annotated_frame, f"{label} {conf:.2f}", (xyxy[0], xyxy[1]-10),
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Log detections to the report if there is at least one and if conditions are met
            if current_detections:
                current_time = time.time()
                detection_str = ", ".join(current_detections)
                if (self.last_yolo_detections != detection_str or 
                    (current_time - self.last_yolo_report_time > 5)):
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    self.report_listbox.insert(tk.END, f"[{timestamp}] YOLO detection: {detection_str}")
                    self.report_listbox.insert(tk.END, "-" * 50)
                    self.last_yolo_report_time = current_time
                    self.last_yolo_detections = detection_str

            # Convert annotated_frame from BGR to RGB, resize for display.
            display_frame = cv2.resize(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB), (400, 300))
            img = ImageTk.PhotoImage(image=Image.fromarray(display_frame))
            self.camera_labels[index].imgtk = img
            self.camera_labels[index].config(image=img)
            time.sleep(0.03)



    def go_back(self):
        """Resets the UI and returns to the login page."""
        for widget in self.root.winfo_children():
            widget.destroy()
        self.__init__(self.root)

    def __del__(self):
        if hasattr(self, 'captures'):
            for cap in self.captures:
                if cap and cap.isOpened():
                    cap.release()

if __name__ == "__main__":
    root = tk.Tk()
    app = SmartMonitoringApp(root)
    root.mainloop()



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
0: 384x640 1 Accident, 455.3ms
Speed: 10.8ms preprocess, 455.3ms inference, 23.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 319.2ms
Speed: 5.3ms preprocess, 319.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 198.1ms
Speed: 2.1ms preprocess, 198.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 196.7ms
Speed: 2.0ms preprocess, 196.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 197.8ms
Speed: 1.9ms preprocess, 197.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 192.5ms
Speed: 1.6ms preprocess, 192.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Accident, 197.4ms
Speed: 1.8ms preprocess, 197.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━